### 1) Mineria de Datos sobre Redes Sociales y Sitios Web
#### 1.1)   Tomando datos de la red social Twitter, al menos 200 tweets, realice un script que recolecte, procese y visualice los datos con las siguientes restricciones:

##### a)Los tres trends más RT del momento.

In [39]:
#Cargamos los paquetes necesarios para entrar a la API de Twitter
import twitter
import json
from prettytable import PrettyTable
from collections import Counter
import pandas as pd

keys = pd.read_json('keys.json')['KEYS']
CONSUMER_KEY = keys['CONSUMER_KEY']
CONSUMER_SECRET = keys['CONSUMER_SECRET']
OAUTH_TOKEN = keys['OAUTH_TOKEN']
OAUTH_TOKEN_SECRET = keys['OAUTH_TOKEN_SECRET']

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

#### Seteamos los WOEID para obtener los resultados 

In [97]:
SPAIN_WOE_ID = 23424950
spain_trends = twitter_api.trends.place(_id=SPAIN_WOE_ID)
hashtags = spain_trends[0]['trends']
trends = map(lambda i: hashtags[i]['name'], range(len(hashtags)))[:3]
pt = PrettyTable()
pt.add_column('Spain Trends', trends)
print pt

+-----------------+
|   Spain Trends  |
+-----------------+
|      Messi      |
|      Atleti     |
| #L6Ncataluña27s |
+-----------------+


#### b )   Listar nombres de usuarios que publicaron con los hashtag de los trends del punto anterior.

In [98]:
# Obtenemos los tweets
CANT = 10
in_spain = set(trends)

tweets = []
for i in range(len(in_spain)):
    results = twitter_api.search.tweets(q=in_spain.pop() , count=CANT)
    tweets += results['statuses']

# Obtenemos los nombres de los users
screen_name = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
screen_name = list(set(screen_name)) #sin repetidos
pt = PrettyTable()
pt.add_column('Users', screen_name )

#mostramos los resultados en una tabla
print pt

+-----------------+
|      Users      |
+-----------------+
|     sadratak    |
|     Mont_Pla    |
|   sihorizontal  |
|  miquelcasellas |
|     mchsccp     |
|  matiascampos99 |
|     zatsuani    |
|   DavidPiri88   |
|   CatSobirana   |
|    ortiiz117    |
|   Meely_Shoue   |
|    Mjvfiel45    |
|    zugrusina    |
|   gerardsoccer  |
| SalvaCasablancs |
|  Nicotorrescab  |
|   JA_Pedraza_   |
|      VotaDN     |
|   dan_casquito  |
|    AnaVeliz92   |
|     Ruth_tvz    |
| JuanManuelLacr1 |
|     femape40    |
|  merceditasmar  |
|  dan_chaparrog  |
|  AgusPedernera6 |
|    AgataName    |
| FrancisJRHowell |
| RahimovKhurshed |
|    seaworld8    |
+-----------------+


#### c) Cuales son las cinco palabras más utilizadas en los tweets del primer item.

In [99]:
text = map( lambda i: tweets[i]['text'], range(len(tweets)))
words = [ word for twt in text for word in twt.split() ]
c = Counter(words)
#print c.most_common()

pt = PrettyTable(field_names=['Words', 'Count'])
map(lambda r: pt.add_row(r), c.most_common()[:5])
print pt

+-------+-------+
| Words | Count |
+-------+-------+
|   RT  |   23  |
|   de  |   19  |
|   la  |   13  |
|   y   |   11  |
|   el  |   10  |
+-------+-------+


#### d) Listar los primeros 10 usuarios con mayor cantidad de seguidores

In [100]:
lista_users_followers = map( lambda i: (tweets[i]['user']['screen_name'],tweets[i]['user']['followers_count']), range(len(tweets)))

lista_users_followers = sorted(lista_users_followers, key=lambda x: x[1], reverse=True)[:10]
users = map((lambda x: x[0]), lista_users_followers)
followers = map((lambda x: x[1]), lista_users_followers)


pt = PrettyTable()
pt.add_column('screen_name', users)
pt.add_column('followers_count', followers)
print pt

+----------------+-----------------+
|  screen_name   | followers_count |
+----------------+-----------------+
|    zatsuani    |       3957      |
|   AgataName    |       2762      |
|  CatSobirana   |       2449      |
| miquelcasellas |       2057      |
|     VotaDN     |       1871      |
|    mchsccp     |       1812      |
|  sihorizontal  |       1137      |
|    Ruth_tvz    |       1100      |
|  Meely_Shoue   |       985       |
|    femape40    |       943       |
+----------------+-----------------+


#### e) Listar la ubicación (o en su defecto time-zone ) del tweet.

In [108]:
users = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
location_or_tz = map( lambda i: tweets[i]['user']['location'] or tweets[i]['user']['time_zone'], range(len(tweets)))

pt = PrettyTable()
pt.add_column('User', users)
pt.add_column('Location or TZ', location_or_tz)
print pt

+-----------------+--------------------------------+
|       User      |         Location or TZ         |
+-----------------+--------------------------------+
|  Nicotorrescab  | Santa Marta , Lomas de Zamora  |
|  dan_chaparrog  |        Bogotá, Colombia        |
|     sadratak    |     Tierra camino de Marte     |
|    AgataName    |         Lampung Tengah         |
| RahimovKhurshed |              None              |
|     mchsccp     |         Hometown Glory         |
|  matiascampos99 |            ROSARIO             |
|   Meely_Shoue   |             Hawaii             |
|  AgusPedernera6 |    Argentina/05-08-15/River    |
|   gerardsoccer  |          México, D.F           |
|   DavidPiri88   |              None              |
|     Mont_Pla    |             arreu              |
|      VotaDN     |             España             |
| JuanManuelLacr1 |              None              |
|    seaworld8    |              None              |
|     femape40    |              None         

#### f) Listar los cinco tweets más populares.


In [121]:
texto_y_rt = map( lambda i: (tweets[i]['text'], tweets[i]['retweet_count'], tweets[i]['user']['id']), range(len(tweets)))

texto_y_rt = sorted(texto_y_rt, key=lambda x: x[1], reverse=True)[:5]
texto = map((lambda x: x[0]), texto_y_rt)
rts = map((lambda x: x[1]), texto_y_rt)


pt = PrettyTable()
pt.add_column('Tweet', texto)
pt.max_width['Tweet'] = 80
pt.add_column('Retweets', rts)
pt.align = 'l'
print pt

+----------------------------------------------------------------------------------+----------+
| Tweet                                                                            | Retweets |
+----------------------------------------------------------------------------------+----------+
| RT @Rdelavarga: Messi viajó por todo USA, jugó 1 partido y medio con la          | 400      |
| Selección, fue papá hace un día, hoy va al banco, ingresa y hace un…             |          |
| RT @Rdelavarga: Messi viajó por todo USA, jugó 1 partido y medio con la          | 400      |
| Selección, fue papá hace un día, hoy va al banco, ingresa y hace un…             |          |
| RT @futmais: GOLAÇO DE FALTA DE NEYMAR! Atléti 1x1 Barça https://t.co/1bInMWZDE1 | 379      |
| RT @ActuFoot_: Thiago &amp; Mateo Messi... http://t.co/Fxyg56ldVR                | 317      |
| RT @AndreHenning: Desculpem aos que ainda forçam em criar uma polêmica, mas      | 197      |
| mesmo com os 5 de hoje, Messi &gt;&gt;

#### g) Listar a los seguidores del autor del tweet más popular